In [14]:
x = 19/104
print(f"{x :.3}")

0.183


In [13]:
x = 19/104
print(f"{x :.2}")

0.18


# Imports

In [33]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from scipy import stats
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, ShuffleSplit
from sklearn.preprocessing import *
from sklearn.decomposition import PCA
from sklearn.feature_selection import RFE, RFECV
from sklearn.linear_model import *
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.tree import DecisionTreeClassifier
from sklearn.utils import resample
from sklearn.datasets import make_regression
from sklearn.metrics import *
from sklearn.compose import make_column_transformer
from sklearn.pipeline import Pipeline, make_pipeline
from xgboost import XGBClassifier, XGBRegressor
from imblearn.datasets import make_imbalance
import statsmodels.api as sm
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import EarlyStopping
from keras.utils import to_categorical
from keras.wrappers.scikit_learn import KerasClassifier

In [2]:
early_stopping_monitor = EarlyStopping(patience=3)

In [3]:
def upsample_classes(data, target):
    
    lst = list(data[target].unique())
    
    classes = []
    for c in lst:
        classes.append(data[data[target]==c])
    
    length = 0
    class_lab = None
    for c in classes:
        if len(c)>length:
            length=len(c)
            class_lab = c
    class_lab = class_lab[target].unique()[0]
    
    regroup = pd.concat(classes)
    maj_class = regroup[regroup[target]==class_lab]

    lst.remove(class_lab)
    
    new_classes=[]
    for i in lst:
        new_classes.append(resample(data[data[target]==i],replace=True, n_samples=len(maj_class)))

    minority_classes = pd.concat(new_classes)
    upsample = pd.concat([regroup[regroup[target]==class_lab],minority_classes])

    return upsample

# continuous first

In [4]:
df = pd.read_csv('diamonds.csv')

In [5]:
df.drop('Unnamed: 0',axis=1,inplace=True)

In [6]:
df.head()

,carat,cut,color,clarity,depth,table,price,x,y,z
0,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43
1,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31
2,0.23,Good,E,VS1,56.9,65.0,327,4.05,4.07,2.31
3,0.29,Premium,I,VS2,62.4,58.0,334,4.20,4.23,2.63
4,0.31,Good,J,SI2,63.3,58.0,335,4.34,4.35,2.75


In [7]:
df['volume'] = df.x*df.y*df.z
df.drop(['x','y','z'],axis=1,inplace=True)

In [8]:
df.head()

,carat,cut,color,clarity,depth,table,price,volume
0,0.23,Ideal,E,SI2,61.5,55.0,326,38.202030
1,0.21,Premium,E,SI1,59.8,61.0,326,34.505856
2,0.23,Good,E,VS1,56.9,65.0,327,38.076885
3,0.29,Premium,I,VS2,62.4,58.0,334,46.724580
4,0.31,Good,J,SI2,63.3,58.0,335,51.917250


In [9]:
for col in df.select_dtypes(include='O').columns:
    df[col] = LabelEncoder().fit_transform(df[col])
    df[col] = df[col].astype(float)

In [10]:
X = df.drop('price',axis=1)
y = df.price
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=14)

In [11]:
model = Sequential()

n_cols = X_train.shape[1]

model.add(Dense(10, activation = 'relu', input_shape = (n_cols, )))
model.add(Dense(10, activation = 'relu'))
model.add(Dense(1))

model.compile(optimizer = 'adam', loss = 'mean_squared_error')

In [12]:
model.fit(X_train, y_train, validation_split=0.2, epochs = 30, callbacks=early_stopping_monitor)

Epoch 1/30
1012/1012 [==============================] - 1s 1ms/step - loss: 14741368.0000 - val_loss: 4408494.5000
Epoch 2/30
1012/1012 [==============================] - 1s 942us/step - loss: 3219980.0000 - val_loss: 2323380.7500
Epoch 3/30
1012/1012 [==============================] - 1s 960us/step - loss: 2819279.5000 - val_loss: 2218742.5000
Epoch 4/30
1012/1012 [==============================] - 1s 945us/step - loss: 2728769.7500 - val_loss: 2144661.2500
Epoch 5/30
1012/1012 [==============================] - 1s 984us/step - loss: 2604113.7500 - val_loss: 2016174.8750
Epoch 6/30
1012/1012 [==============================] - 1s 940us/step - loss: 2506410.2500 - val_loss: 1929031.0000
Epoch 7/30
1012/1012 [==============================] - 1s 941us/step - loss: 2413221.5000 - val_loss: 1857556.7500
Epoch 8/30
1012/1012 [==============================] - 1s 944us/step - loss: 2343714.2500 - val_loss: 1808790.7500
Epoch 9/30
1012/1012 [==============================] - 1s 940us/step - l

In [13]:
preds = model.predict(X_test)

In [14]:
print(r2_score(y_test,preds))
print(mean_absolute_error(y_test,preds))

0.9017847694105239
763.2608433006293


In [15]:
model2 = Sequential()

model2.add(Dense(200, activation='relu', input_shape=(n_cols,)))
model2.add(Dense(200, activation='relu'))
model2.add(Dense(200, activation='relu'))
model2.add(Dense(1))

model2.compile(optimizer = 'adam', loss = 'mean_squared_error')

model2.fit(X_train, y_train, validation_split = 0.2, epochs = 30, callbacks = early_stopping_monitor)

Epoch 1/30
1012/1012 [==============================] - 2s 2ms/step - loss: 3486223.0000 - val_loss: 1547608.2500
Epoch 2/30
1012/1012 [==============================] - 1s 1ms/step - loss: 2161514.7500 - val_loss: 1234288.5000
Epoch 3/30
1012/1012 [==============================] - 1s 1ms/step - loss: 1582822.6250 - val_loss: 1116687.5000
Epoch 4/30
1012/1012 [==============================] - 1s 1ms/step - loss: 1258404.0000 - val_loss: 1132536.7500
Epoch 5/30
1012/1012 [==============================] - 1s 1ms/step - loss: 1205241.1250 - val_loss: 1112664.3750
Epoch 6/30
1012/1012 [==============================] - 1s 1ms/step - loss: 1154736.5000 - val_loss: 1111808.6250
Epoch 7/30
1012/1012 [==============================] - 1s 1ms/step - loss: 1140422.6250 - val_loss: 1076430.8750
Epoch 8/30
1012/1012 [==============================] - 1s 1ms/step - loss: 1141080.2500 - val_loss: 1046665.4375
Epoch 9/30
1012/1012 [==============================] - 1s 1ms/step - loss: 1144466.3750

In [16]:
preds2 = model2.predict(X_test)

In [17]:
print(r2_score(y_test,preds2))
print(mean_absolute_error(y_test,preds2))

0.9674075284004763
380.4542182806381


# discrete binary

In [143]:
df3=df.copy()

In [144]:
df3['price_bin']=pd.cut(df3.price,bins=2,labels=[0,1])

In [145]:
def run_binary_nn(data, target):
    
    y=data[target]
    X=data.drop(target,axis=1)
    
    X=scale(X)
    
    X_train, X_test, y_train, y_test = train_test_split(X,y)
    
    y_train=to_categorical(y_train)
    
    model=Sequential()
    model.add(Dense(12, input_dim = data.drop(target,axis=1).shape[1], activation='relu'))
    model.add(Dense(8, activation='relu'))
    model.add(Dense(len(data[target].unique()), activation='sigmoid'))
    
    model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
    
    model.fit(X_train,y_train,epochs=15,batch_size=10)
    
    y_pred = model.predict(X_test)
    y_pred = np.argmax(y_pred,axis=1)
    
    print(classification_report(y_test,y_pred))

In [147]:
run_binary_nn(df3,'price_bin')

Epoch 1/15
4046/4046 [==============================] - 4s 953us/step - loss: 0.0689 - accuracy: 0.9789
Epoch 2/15
4046/4046 [==============================] - 4s 882us/step - loss: 0.0125 - accuracy: 0.9963
Epoch 3/15
4046/4046 [==============================] - 4s 954us/step - loss: 0.0090 - accuracy: 0.9971
Epoch 4/15
4046/4046 [==============================] - 3s 829us/step - loss: 0.0075 - accuracy: 0.9971
Epoch 5/15
4046/4046 [==============================] - 3s 783us/step - loss: 0.0068 - accuracy: 0.9977
Epoch 6/15
4046/4046 [==============================] - 3s 854us/step - loss: 0.0064 - accuracy: 0.99770s - loss: 0.006
Epoch 7/15
4046/4046 [==============================] - 4s 937us/step - loss: 0.0055 - accuracy: 0.9978
Epoch 8/15
4046/4046 [==============================] - 4s 994us/step - loss: 0.0057 - accuracy: 0.9978
Epoch 9/15
4046/4046 [==============================] - 4s 1ms/step - loss: 0.0052 - accuracy: 0.9980: 0s - loss: 0.0051 - accuracy: 0.
Epoch 10/15
4046

# discrete mmulti-class

In [148]:
def run_multi_class_nn(data, target):
    
    y=data[target]
    X=data.drop(target,axis=1)
    
    X=scale(X)
    
    X_train, X_test, y_train, y_test = train_test_split(X,y)
    
    y_train=to_categorical(y_train)
    
    model=Sequential()
    model.add(Dense(12, input_dim = data.drop(target,axis=1).shape[1], activation='relu'))
    model.add(Dense(8, activation='relu'))
    model.add(Dense(len(data[target].unique()), activation='softmax'))
    
    model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
    
    model.fit(X_train,y_train,epochs=15,batch_size=10)
    
    y_pred = model.predict(X_test)
    y_pred = np.argmax(y_pred,axis=1)
    
    print(classification_report(y_test,y_pred))

In [149]:
run_multi_class_nn(df2,'price_bin')

Epoch 1/15
4046/4046 [==============================] - 3s 826us/step - loss: 0.3243 - accuracy: 0.8640
Epoch 2/15
4046/4046 [==============================] - 3s 837us/step - loss: 0.2416 - accuracy: 0.8922
Epoch 3/15
4046/4046 [==============================] - 3s 780us/step - loss: 0.2310 - accuracy: 0.8947
Epoch 4/15
4046/4046 [==============================] - 3s 832us/step - loss: 0.2222 - accuracy: 0.9002
Epoch 5/15
4046/4046 [==============================] - 4s 921us/step - loss: 0.2102 - accuracy: 0.9091
Epoch 6/15
4046/4046 [==============================] - 4s 904us/step - loss: 0.1975 - accuracy: 0.9178
Epoch 7/15
4046/4046 [==============================] - 3s 795us/step - loss: 0.1891 - accuracy: 0.9210
Epoch 8/15
4046/4046 [==============================] - 4s 949us/step - loss: 0.1843 - accuracy: 0.9223
Epoch 9/15
4046/4046 [==============================] - 5s 1ms/step - loss: 0.1810 - accuracy: 0.9235
Epoch 10/15
4046/4046 [==============================] - 3s 820us/

***

In [107]:
df2 = pd.read_csv('sample.csv')

In [108]:
df2.head()

,Unnamed: 0,carat,cut,color,clarity,depth,table,price,price_bin,volume
0,0,0.23,Ideal,E,SI2,61.5,55.0,326.0,0.0,38.202030
1,1,0.21,Premium,E,SI1,59.8,61.0,326.0,0.0,34.505856
2,2,0.23,Good,E,VS1,56.9,65.0,327.0,0.0,38.076885
3,3,0.29,Premium,I,VS2,62.4,58.0,334.0,0.0,46.724580
4,4,0.31,Good,J,SI2,63.3,58.0,335.0,0.0,51.917250


In [109]:
df2['price_bin']=pd.cut(df2.price,bins=4,labels=[0,1,2,3])

In [110]:
df2.drop(['Unnamed: 0', 'price'], axis=1, inplace=True)

In [111]:
for col in df2.select_dtypes(include='O').columns:
    df2[col] = LabelEncoder().fit_transform(df2[col])

In [128]:
X = df2.drop(['price_bin'],axis=1)
y = df2.price_bin

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.30,random_state=14)

In [135]:
y_train = to_categorical(y_train)
#y_test = to_categorical(y_test)

count_classes = y_train.shape[1]
print(count_classes)

4


In [136]:
model3 = Sequential()

n_cols_3 = X_train.shape[1]

model3.add(Dense(250, activation='relu', input_shape = (n_cols_3,)))
model3.add(Dense(250, activation='relu'))
model3.add(Dense(250, activation='relu'))
model3.add(Dense(4, activation='sigmoid'))

In [137]:
model3.compile(optimizer='adam', loss='categorical_crossentropy',metrics=['accuracy'])

In [138]:
model3.fit(X_train, y_train, epochs=10, validation_split= 0.25, callbacks=early_stopping_monitor)

Epoch 1/10


ValueError: in user code:

    /Users/josephcohen/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:571 train_function  *
        outputs = self.distribute_strategy.run(
    /Users/josephcohen/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:951 run  **
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /Users/josephcohen/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:2290 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /Users/josephcohen/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:2649 _call_for_each_replica
        return fn(*args, **kwargs)
    /Users/josephcohen/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:533 train_step  **
        y, y_pred, sample_weight, regularization_losses=self.losses)
    /Users/josephc

In [134]:
y_pred = model.predict(X_test)
y_pred = np.argmax(y_pred,axis=1)

ValueError: in user code:

    /Users/josephcohen/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:1147 predict_function  *
        outputs = self.distribute_strategy.run(
    /Users/josephcohen/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:951 run  **
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /Users/josephcohen/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:2290 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /Users/josephcohen/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:2649 _call_for_each_replica
        return fn(*args, **kwargs)
    /Users/josephcohen/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:1122 predict_step  **
        return self(x, training=False)
    /Users/josephcohen/opt/anaconda3/lib/p

In [125]:
y_test.shape

(16182,)

In [103]:
print(classification_report(y_test,y_pred))

ValueError: Classification metrics can't handle a mix of multilabel-indicator and binary targets

In [77]:
pred_train= model3.predict(X_train)
scores = model3.evaluate(X_train, y_train, verbose=0)
 
pred_test= model3.predict(X_test)
scores2 = model3.evaluate(X_test, y_test, verbose=0)    

ValueError: in user code:

    /Users/josephcohen/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:941 test_function  *
        outputs = self.distribute_strategy.run(
    /Users/josephcohen/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:951 run  **
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /Users/josephcohen/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:2290 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /Users/josephcohen/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:2649 _call_for_each_replica
        return fn(*args, **kwargs)
    /Users/josephcohen/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:912 test_step  **
        y, y_pred, sample_weight, regularization_losses=self.losses)
    /Users/josephcoh

In [ ]:
cl

In [ ]:
prds3 = []
for i in list(preds3.reshape(-1,1)):
    prds3.append(int(i))
    
y_t_3 = []
for i in list(y_test.reshape(-1,1)):
    y_t_3.append(int(i))